# 1 Intro

In this notebook, we learn how to use R to perform a t-test. I assume that you know what a t-test is, and therefore will only focus on the R coding part. Nevertheless, I will briefly review the concept of t-test.

T-test is a statistical hypothesis test for evaluating means of one or two populations. There are three kinds of t-tests for mean comparisons.

1. A *one-sample t-test* is used to decide if the population mean is equal to a specific value or not, e.g., whether the mean heart rate of a group of people is equal to 70 or not.

2. A *two-sample t-test* is used to decide if the population means for two different groups are equal or not, e.g., whether the mean heart rates for two groups of people are the same or not.

3. A *paired t-test* is used to decide if the difference between paired measurements for a population is zero or not, e.g., whether mean difference in heart rate for a group of people before and after exercise is zero or not.

To perform a t-test, you can follow the below steps.

1. Define the hypothesis you are going to test (null hypothesis $H_0$ and alternative hypothesis $H_a$), and specify an acceptable risk of drawing a faulty conclusion, i.e., the type-I error rate (or significance level), $\alpha$ (e.g., $\alpha=0.05$).

2. Calculate a test statistic from your data, and compare it to a theoretical value (critical value) from a t-distribution. Alternatively, you can calculate the p-value corresponding to the test statistics. Depending on the outcome, you either reject or fail to reject your null hypothesis.

See [here](https://www.jmp.com/en_ca/statistics-knowledge-portal/t-test.html) for a discussion on the three t-tests including an excellent comparison table. Most of my above discussion are drawn from there.

**Note 1**: t-test is also commonly used in linear regression analysis to test whether a coefficient/slope of a regressor (i.e., a predictor or a explanatory variable) is equal to zero. You will see this when we discuss how to run linear regression using R.

**Note 2**: People often believe that t-test requires the assumption that the data follows a normal distribution. However, by the central limit theorem, sample mean of moderately large samples (a size above 30) is often well-approximated by a normal distribution, even if the individual data are not normally distributed. In fact, that's often good enough for t-test, so for large samples, the t-test can be applied without the data being strictly from a normal distribution ([Lumley et. al., 2002](https://pubmed.ncbi.nlm.nih.gov/11910059/)). On the other hand, it is still important to ensure that other assumptions of the t-test are met: 1) the sample data are randomly sampled from a population; 2) for two-sample t-test, the variability of the data in each group is similar. (However, if samples are from normal distribution and the sizes of the two groups are the same, t-test is highly robust to the presence of unequal variances ([Markowski & Markowski, 1990](https://doi.org/10.2307/2684360))).

# 2 One-sample t-test

Let's generate some data for the t-test exercise. We will generate 300 independent random normals with mean = 70 and standard deviation 5.

In [ ]:
# set random seed so you can reproduce the result
set.seed(1)

In [ ]:
# 300 normal with mean = 70 and sd = 5
heart_rate <- rnorm(n = 300, mean = 70, sd = 5)
heart_rate[1:10]

## 2.1 A two-sided one-sample t-test

Let's test whether the population mean is equal to 71. Our null hypothesis is $H_0: \mu = 71$ and alternative hypothesis is $H_a: \mu \neq 71$. We choose significance level $\alpha = 0.05$.

As a practice, let's first calculate the t test statistic using the formula.

$$t = \frac{\bar{x}-\mu_0}{s/\sqrt{n}},$$

where $\bar{x}$ is sample mean, $\mu_0$ is population mean under null, $s$ is sample standard deviation, and $n$ is number of samples.

In [ ]:
# calculate t statistic
x <- heart_rate
n <- length(heart_rate)
t <- (mean(x) - 71) / (sd(x) / sqrt(n))
t

The degree of freedom in this one sample test is $n-1 = 299$. Since t-distribution is symmetric, we can find the critical value for this two-sided test by find the 97.5% quantile of a t-distribution with degree of freedom 299.

In [ ]:
# t-distribution document, https://stat.ethz.ch/R-manual/R-devel/library/stats/html/TDist.html
critical_value <- qt(p = 0.975, df = 299)
critical_value

The critical values are $\pm1.968$. Note that this number is close to the one that you will get from a standard normal distribution, $\pm1.96$. This is expected. At a degree of freedom 299, the t-distribution is approximately a standard normal.

Since t-distribution is symmetric, we can compare the absolute value of our t statistic to the positive critical value. Since $|t| > 1.968$, we reject the null hypothesis.

The t-distribution (under the null hypothesis), t statistic, and critical values are illustrated in the below plot. Each pink region in the plot has a mass of 0.025.

In [ ]:
# plot the t distribution on the domain [-4,4]
curve(dnorm(x),
      xlim = c(-4, 4),
      main = 't-distriubtion (df=299)',
      yaxs = 'i',
      xlab = 't statistic',
      ylab = '',
      lwd = 2,    #line width
      axes = FALSE)

# add x-axis
axis(1,
     at = c(-4, -2.99, 0, -1.968, 1.968, 4),
     padj = 0.5,
     labels = c('', expression(t==-2.99), 0, "-1.968", "1.968", ''))

# shade rejection region in left tail
polygon(x = c(-4, seq(-4, -1.968, 0.01), -1.968),
        y = c(0, dt(seq(-4, -1.968, 0.01), 299), 0),
        col = 'deeppink')

# shade rejection region in right tail
polygon(x = c(1.968, seq(1.968, 4, 0.01), 4),
        y = c(0, dt(seq(1.968, 4, 0.01), 299), 0),
        col = 'deeppink')

Alternatively, we can calculate the p-value for the t statistic.

In [ ]:
p_value <- 2 * pt(q = t, df = 299, lower.tail = TRUE)
p_value

Since the p-value is less than 0.05, we reject the null hypothesis, reaching the same conclusion as using the critical value approach.

Now, R conveniently provides us a t-test function, `t.test()`, so we don't need to do the calculation like above, and we can simply call the `t.test()` function to perform a t-test.

In [ ]:
# t.test() document, https://stat.ethz.ch/R-manual/R-devel/library/stats/html/t.test.html
t.test(x = heart_rate, mu = 71, alternative = "two.sided")

Note that the t statistic, the degree of freedom and the p-value all match our manual calculation.

## 2.2. A one-side one-sample t-test

Now what if our hypothesis is $H_0: \mu <= 71$ and $H_a: \mu > 71$. We again choose significance level $\alpha = 0.05$. This is a one-sided test. We again call the `t.test()` function. We just need to set the correct argument.

In [ ]:
t.test(x = heart_rate, mu = 71, alternative = "great")

This time, the p-value is greater than 0.05 so we fail to reject the null. (Note that as expected the t statistic and degree of freedom is the same as before.)

## Exercise

Draw 200 samples independently from a uniform (0, 10) distribution, and test the hypothesis that the population mean is 5.

In [ ]:
# your code here

# 3 Two-sample t-test

Start by generating some data.

In [ ]:
# the first group is the same as the one from the previous draw
heart_rate_group1 <- heart_rate

# generate data for group 2
heart_rate_group2 <- rnorm(n = 300, mean = 71, sd = 5)

Let's test $H_0: \mu_1 = \mu_2$ and $H_a: \mu_1 \neq \mu_2$. Again, we choose the significance level $\alpha = 0.05$.

In [ ]:
t.test(x = heart_rate_group1, y = heart_rate_group2, alternative = "two.sided")

The p-value is greater than 0.05 so we fail to reject the null hypothesis although it's a close call.

The `t.test()` function has another argument `var.equal`. It is "a logical variable indicating whether to treat the two variances [from the two groups] as being equal. If TRUE then the pooled variance is used to estimate the variance otherwise the Welch (or Satterthwaite) approximation to the degrees of freedom is used."

The default value for `var.equal` is `FALSE`. We could set it to `TRUE` in our case as we know the variance from the two groups are the same.

In [ ]:
t.test(x = heart_rate_group1, y = heart_rate_group2, alternative = "two.sided", var.equal = TRUE)

Notice the change in degree of freedom calculation. Again, we fail to reject the null hypothesis.

## Question

We generated the data so we know the population means for group 1 and 2 are 70 and 71 respectively, but how come our t-test fails to reject the null hypothesis that the two means are the same?

# 4 Paired t-test

Start again by generating some data.

In [ ]:
# heart rate before exercise (same data as the first draw)
heart_rate_before_ex <- heart_rate

# heart rate after exercise (same data as the group 2 before)
heart_rate_after_ex <- heart_rate_group2

We will test $H_0: \mu_d = 0$ and $H_a: \mu_d \neq 0$, where $\mu_d$ is population mean of difference. Keep significance level $\alpha = 0.05$.

**Note**: In a paired t-test, we test whether the mean of differences is zero. In the previous two-sample t-test, we test whether difference in means is zero.

Now apply the paired t-test.

In [ ]:
t.test(x = heart_rate_before_ex, y = heart_rate_after_ex, alternative = "two.sided", paired = TRUE)

We fail to reject null.

Note that the above paired t-test is equivalent to the below one sample t-test on difference.

In [ ]:
dif <- heart_rate_before_ex - heart_rate_after_ex
t.test(x = dif, mu = 0, alternative = "two.sided")

# References

1. The t-test, https://www.jmp.com/en_ca/statistics-knowledge-portal/t-test.html
2. Markowski, C. A., & Markowski, E. P. (1990). Conditions for the Effectiveness of a Preliminary Test of Variance. The American Statistician, 44(4), 322–326. https://doi.org/10.2307/2684360
3. Lumley, T., Diehr, P., Emerson, S., & Chen, L. (2002). The importance of the normality assumption in large public health data sets. Annual review of public health, 23, 151–169. https://doi.org/10.1146/annurev.publhealth.23.100901.140546